In [110]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [111]:
dataset_path = 'train.csv'

In [112]:
df = pd.read_csv(dataset_path)
test_df = pd.read_csv('test.csv')

In [113]:
df.set_index('Id', inplace=True)
test_df.set_index('Id', inplace=True)
#test_df


In [114]:
df.loc[:,['Price']]

,Price
Id,
14038,184966.930730
15053,300009.450063
4765,220925.908524
5809,175616.227217
10783,150226.531644
...,...
77,196684.316040
6159,189050.289571
5123,159143.805370


In [115]:
#df.set_index('Id', inplace = True)
df.tail(10)


,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
Id,,,,,,,,,,,,,,,,,,,
15233,1,3.0,57.373889,39.688445,5.0,2,5.0,1962,0.036270,B,B,6,1318,1,200.0,1,1,B,163807.542987
11553,3,3.0,101.622794,51.628780,12.0,18,17.0,2000,0.265089,B,B,37,5288,0,1937.0,3,2,B,212883.501504
14333,150,3.0,78.249637,49.385096,10.0,5,16.0,1976,0.300323,B,B,52,10311,6,NaN,1,9,B,342418.758888
48,13,3.0,80.631333,48.899083,10.0,11,14.0,1999,0.090799,B,B,74,19083,2,NaN,5,15,B,350684.628560
4158,23,2.0,64.864198,NaN,1.0,17,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,171323.515653
77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040
6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A,189050.289571
5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,159143.805370
5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808


In [116]:
#print(df.loc[df['HouseYear'] > 2019])
#df.dtypes
df.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [117]:
df['Rooms'].value_counts()

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [118]:
df['DistrictId'].value_counts()

27     851
1      652
23     565
6      511
9      294
      ... 
199      1
205      1
174      1
209      1
207      1
Name: DistrictId, Length: 205, dtype: int64

In [119]:
df.loc[df['DistrictId'] == 27 , ['Rooms', 'Square', 'KitchenSquare', 'Price', 'Price']].tail(5)

,Rooms,Square,KitchenSquare,Price,Price
Id,,,,,
10463,3.0,49.574590,1.0,145025.095667,145025.095667
6612,2.0,59.460685,0.0,135784.320930,135784.320930
5071,2.0,60.055527,12.0,190325.071232,190325.071232
16159,1.0,38.968338,1.0,95529.569405,95529.569405
5123,1.0,47.939008,1.0,159143.805370,159143.805370


In [120]:
district_prices_level = df.loc[:,['DistrictId', 'Price']].groupby('DistrictId',as_index=True).mean()
#tt.loc[203,['DistrictId','Price']][1]
d1 = df.loc[:,['DistrictId', 'Price']].groupby('DistrictId',as_index=True).mean()
d2 = df.loc[:,['DistrictId', 'Square']].groupby('DistrictId',as_index=True).mean()
dist=d1.merge(d2, on='DistrictId').copy()

Hl_mean = df.loc[:,['DistrictId', 'Healthcare_1']].groupby('DistrictId',as_index=True).mean()
#district_sq_m
dist['av_m2'] = dist['Price']/dist['Square']
dist

,Price,Square,av_m2
DistrictId,,,
0,195295.590075,54.549887,3580.128227
1,187805.401737,59.147410,3175.209200
2,243688.618402,53.870293,4523.617804
3,179905.016194,50.138363,3588.170925
4,300096.687440,66.822950,4490.922499
...,...,...,...
202,394150.861857,68.245487,5775.486120
205,220501.566180,43.226985,5101.016598
207,426186.409334,76.780960,5550.678339


In [121]:
for i in dist.iterrows():
    #print(i[1][2])
    #print(i[0])
    #print(Hl_mean.loc[154,'Healthcare_1'])
    #df.loc[df['Healthcare_1'].isna(), 'Healthcare_1'] = Hl_mean.loc[int(i[1][1]),'Healthcare_1']
    df.loc[df['DistrictId'] == i[0], 'fff'] = dist.loc[i[0],'av_m2']
    #print(int(df.loc[i[0]][0]))
    #df.loc[int(df.loc[i[0]][0]), 'Square']
    #print(Hl_mean.loc[int(i[1][1]),'Healthcare_1'])

In [122]:
for i in dist.iterrows():
    #print(i[1][2])
    #print(i[0])
    #print(Hl_mean.loc[154,'Healthcare_1'])
    #df.loc[df['Healthcare_1'].isna(), 'Healthcare_1'] = Hl_mean.loc[int(i[1][1]),'Healthcare_1']
    test_df.loc[test_df['DistrictId'] == i[0], 'fff'] = dist.loc[i[0],'av_m2']

In [123]:
#df.loc[df['DistrictId'] == 35, 'fff'] = 4
#df.loc[5809, :]

In [124]:
df.head(5)

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,fff
Id,,,,,,,,,,,,,,,,,,,,
14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730,4366.167054
15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,4241.295364
4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,4766.966334
5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,2957.990008
10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,3878.845206


In [125]:
feature_bin_names = ['Ecology_2', 'Ecology_3', 'Shops_2']

#df[feature_bin_names] = df[feature_bin_names].replace({'A':0, 'B':1})
#test_df[feature_bin_names] = test_df[feature_bin_names].replace({'A':0, 'B':1})

In [126]:
df.loc[df['LifeSquare'].isna(), ['Rooms', 'Square', 'KitchenSquare', 'Price']].tail(5)

,Rooms,Square,KitchenSquare,Price
Id,,,,
16159,1.0,38.968338,1.0,95529.569405
2996,2.0,64.994245,1.0,200332.511658
15623,2.0,55.620092,0.0,115797.553006
4158,2.0,64.864198,1.0,171323.515653
5123,1.0,47.939008,1.0,159143.805370


In [127]:
df['KitchenSquare'].mean()

6.2733

In [128]:
df['Square'].mean()

56.31577519793556

In [129]:
#11.13% занимает кухня

In [130]:
df.isna().sum()

DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
fff                 0
dtype: int64

In [131]:
for i in df.loc[df['Healthcare_1'].isna(), ['Healthcare_1', 'DistrictId']].iterrows():
    #print(i[1][1])
    print(Hl_mean.loc[154,'Healthcare_1'])
    #df.loc[df['Healthcare_1'].isna(), 'Healthcare_1'] = Hl_mean.loc[int(i[1][1]),'Healthcare_1']
    #df.iloc[i[0], 'Healthcare_1'] = Hl_mean.loc[int(i[1][1]),'Healthcare_1']
    #print(df.loc[i[0], 'Healthcare_1'])
    #df.loc[i[0], 'Healthcare_1'] = Hl_mean.loc[int(i[1][1]),'Healthcare_1']
    #print(Hl_mean.loc[int(i[1][1]),'Healthcare_1'])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [132]:
df

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,fff
Id,,,,,,,,,,,,,,,,,,,,
14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730,4366.167054
15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,4241.295364
4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,4766.966334
5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,2957.990008
10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,3878.845206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040,4940.495992
6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A,189050.289571,3787.535658
5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,159143.805370,2715.704541


In [133]:
#for i in df.loc[df['Healthcare_1'].isna(), ['Healthcare_1', 'DistrictId']].iterrows(): print(int(i[1][1])) df.loc[df['Healthcare_1'].isna(), 'Healthcare_1'] = Hl_mean.loc[int(i[1][1]),'Healthcare_1']

#i[1][0] = Hl_mean
#print(Hl_mean.loc[int(i[1][1]),'Healthcare_1'])

In [134]:
df.loc[df['Healthcare_1'].isna(), 'Healthcare_1'] = df['Healthcare_1'].median()


In [135]:
test_df

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,fff
Id,,,,,,,,,,,,,,,,,,,
725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B,2957.990008
15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,3272.322491
5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B,7735.224377
15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B,4063.439258
14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A,2715.704541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,B,B,36,5992,0,NaN,1,1,B,3997.872947
4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,B,B,1,264,0,NaN,0,1,B,3175.209200
5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,B,B,74,19083,2,NaN,5,15,B,4061.853520


In [136]:
test_df.loc[test_df['Healthcare_1'].isna(), 'Healthcare_1'] = test_df['Healthcare_1'].median()
test_df.loc[test_df['fff'].isna(), 'fff'] = test_df['fff'].median()

In [137]:
df.isna().sum()

DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1        0
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
fff                 0
dtype: int64

df['Cat_'] = train_df['Price'] / train_df['Rooms']

In [138]:
df.loc[df['Rooms'] == 5 , ['Rooms', 'Square', 'KitchenSquare', 'Price', 'LifeSquare']].tail(50)

,Rooms,Square,KitchenSquare,Price,LifeSquare
Id,,,,,
4282,5.0,95.617533,7.0,479525.147323,62.377052
1243,5.0,108.057398,8.0,310305.074733,82.723825
5548,5.0,275.645284,26.0,455264.882666,233.949309
8901,5.0,121.577423,1.0,455446.393758,114.913843
6358,5.0,42.003293,0.0,112693.641769,NaN
7594,5.0,39.906082,1.0,206148.845716,39.394024
13434,5.0,183.382273,15.0,622888.825328,100.985505
4070,5.0,114.313752,9.0,499830.937577,88.492650
2879,5.0,118.871952,10.0,496026.671162,69.023509


In [139]:
df.loc[(df['Square'] > 500), 'Square'] = df['Square'].median()
df.loc[(df['Square'] < 10, 'Square')] = df['Square'].median()
test_df.loc[(test_df['Square'] > 500), 'Square'] = test_df['Square'].median()
test_df.loc[(test_df['Square'] < 10, 'Square')] = test_df['Square'].median()

In [140]:
df

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,fff
Id,,,,,,,,,,,,,,,,,,,,
14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730,4366.167054
15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,4241.295364
4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,4766.966334
5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,2957.990008
10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,3878.845206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040,4940.495992
6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,900.0,0,5,A,189050.289571,3787.535658
5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,900.0,0,0,A,159143.805370,2715.704541


In [141]:
#df['Shops_2_A'] = 1 if df['Shops_2'] == 'A' else 0
#test_df['RoomSquare'] = test_df['Rooms'] / test_df['Square']
#df

In [142]:
feat_object = list(df.select_dtypes(include='object').columns)
print(feat_object)
for col in feat_object:
    
    df_temp = pd.get_dummies(df[col])

    df_temp.columns = [col + '_' + c for c in df_temp]
    
    df.drop(
        labels=col,
        axis=1,
        inplace=True
    )
    
    df = pd.concat(
        objs=(df, df_temp),
        axis=1
    )
    
    del df_temp

['Ecology_2', 'Ecology_3', 'Shops_2']


In [143]:
for col in feat_object:
    
    df_temp = pd.get_dummies(test_df[col])

    df_temp.columns = [col + '_' + c for c in df_temp]
    
    test_df.drop(
        labels=col,
        axis=1,
        inplace=True
    )
    
    test_df = pd.concat(
        objs=(test_df, df_temp),
        axis=1
    )
    
    del df_temp

In [144]:
df.loc[(df['KitchenSquare'] < 3), 'KitchenSquare'] = df['KitchenSquare'].median()
test_df.loc[(test_df['KitchenSquare'] < 3), 'KitchenSquare'] = test_df['KitchenSquare'].median()
#df.loc[(df['KitchenSquare'] > 0), 'KitchenSquare'] = 1
#df.loc[(df['KitchenSquare'] < 1), 'KitchenSquare'] = 0
#test_df.loc[(test_df['KitchenSquare'] > 0), 'KitchenSquare'] = 1
#test_df.loc[(test_df['KitchenSquare'] < 1), 'KitchenSquare'] = 0
test_df.loc[(test_df['Rooms'] < 1), 'Rooms'] = 1
test_df.loc[(test_df['Rooms'] > 10), 'Rooms'] = df['Rooms'].median()
df.loc[(df['Rooms'] < 1), 'Rooms'] = 1
df.loc[(df['Rooms'] > 10), 'Rooms'] = df['Rooms'].median()

In [145]:

df.loc[(df['HouseYear'] == 4968)]

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Helthcare_2,Shops_1,Price,fff,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
Id,,,,,,,,,,,,,,,,,,,,,
11607,147,2.0,44.791836,28.360393,5.0,4,9.0,4968,0.319809,25,...,5,8,243028.603096,5136.595338,0,1,0,1,0,1


In [146]:
#df.drop([11607], inplace=True)
#df.loc[(df['HouseYear'] == 4968), 'HouseYear'] =  1968

In [147]:
df.loc[(df['HouseYear'] == 1968)]

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Helthcare_2,Shops_1,Price,fff,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
Id,,,,,,,,,,,,,,,,,,,,,
4765,53,2.0,44.947953,29.197612,6.0,8,12.0,1968,0.049637,34,...,1,3,220925.908524,4766.966334,0,1,0,1,0,1
3918,41,2.0,48.077041,28.835117,5.0,5,9.0,1968,0.175055,36,...,1,0,198183.076920,4241.295364,0,1,0,1,0,1
10337,28,2.0,43.771830,33.242167,5.0,3,9.0,1968,0.118537,30,...,1,0,203425.396064,4345.689721,0,1,0,1,0,1
12087,2,3.0,60.159406,46.354527,5.0,4,9.0,1968,0.309479,35,...,0,6,317296.305300,4523.617804,0,1,0,1,0,1
14264,119,2.0,45.222690,33.485943,5.0,6,9.0,1968,0.000995,42,...,1,5,186508.208565,4187.115637,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,27,1.0,29.386944,16.820321,5.0,3,4.0,1968,0.211401,9,...,0,1,121716.142598,2715.704541,0,1,0,1,0,1
12992,75,2.0,47.917740,29.839365,5.0,7,5.0,1968,0.307467,30,...,2,5,184252.980508,4208.013270,0,1,1,0,0,1
12913,8,2.0,48.406418,30.717760,5.0,13,12.0,1968,0.458174,26,...,1,5,212792.292586,4002.262120,0,1,1,0,0,1


In [148]:
df.loc[(df['HouseYear'] > 2020), 'HouseYear'] = 2020
#df.loc[(df['HouseYear'] < 2020), 'HouseYear'] = df['HouseYear'].median()
test_df.loc[(test_df['HouseYear'] > 2020), 'HouseYear'] = 2020
#test_df.loc[(test_df['HouseYear'] < 2020), 'HouseYear'] = test_df['HouseYear'].median()

In [149]:
df.loc[df['LifeSquare'].isna(), 'LifeSquare'] = df['Square'].median()-df['Square'].median()*0.13
test_df.loc[test_df['LifeSquare'].isna(), 'LifeSquare'] = test_df['Square'].median()-test_df['Square'].median()*0.13

In [150]:
#df.loc[df['LifeSquare'].isna(), 'LifeSquare'] = df['LifeSquare'].median()
#test_df.loc[test_df['LifeSquare'].isna(), 'LifeSquare'] = test_df['LifeSquare'].median()

In [151]:
target_name = ['Price']
feature_names = [
             'DistrictId',
             'Rooms',
             'Square',
             'LifeSquare',
             'KitchenSquare',
             'Floor',
             'HouseFloor',
             'HouseYear',
             'Ecology_1',
            # 'Ecology_2',
             #'Ecology_3',
             'Social_1',
             'Social_2',
             'Social_3',
            'Healthcare_1',
             'Helthcare_2',
             'Shops_1',
           # 'Shops_2',
    'Ecology_2_A',
    'Ecology_2_B',
    'Ecology_3_A',
    'Ecology_3_B',
    'Shops_2_A',
    'Shops_2_B',
    'fff',
#             'Price'
#             'DistrictSize',
#             'PriceOneRoom'
#             'PriceOneRoomByDistrict',
#             'RoomSquare'
#             'Floor_cat',
#             'HouseFloor_cat',
#             'StoreysFloor',
#             'PriceOneRoomByStoreysFloor'
]

In [152]:
X = df[feature_names]
y = df[target_name]


X_test = test_df[feature_names]

In [153]:
X_test

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Healthcare_1,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B,fff
Id,,,,,,,,,,,,,,,,,,,,,
725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,11,...,900.0,0,0,0,1,0,1,0,1,2957.990008
15856,74,2.0,69.263183,46.043098,6.0,6,1.0,1977,0.075779,6,...,900.0,0,2,0,1,0,1,0,1,3272.322491
5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,30,...,4702.0,5,5,0,1,0,1,0,1,7735.224377
15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,23,...,900.0,3,3,0,1,0,1,0,1,4063.439258
14275,27,1.0,47.527111,43.387569,6.0,17,17.0,2017,0.072158,2,...,900.0,0,0,0,1,0,1,1,0,2715.704541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,36,...,900.0,1,1,0,1,0,1,0,1,3997.872947
4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,1,...,900.0,0,1,0,1,0,1,0,1,3175.209200
5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,74,...,900.0,5,15,0,1,0,1,0,1,4061.853520


In [154]:
from sklearn import ensemble
#from sklearn.kernel_ridge import KernelRidge

#clf = ensemble.GradientBoostingRegressor(n_estimators=342, max_depth=6, min_samples_split=2,
#                                           learning_rate=0.04, loss='ls', random_state=42)

#kf = KFold(n_splits=5, random_state=42, shuffle=True)


#cv_score = cross_val_score(clf, X, y, scoring='r2', cv=kf, n_jobs = -1)
#print('R2: {:.5f}'.format(cv_score.mean()))

from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': np.arange(40, 90),
               'min_samples_split': np.arange(2, 7),
               'max_depth': np.arange(4, 8),
              'learning_rate':[0.1]}]

clf = GridSearchCV(estimator=ensemble.GradientBoostingRegressor(random_state=42),
                   param_grid=parameters, scoring='r2',cv=3, n_jobs=-1)

In [155]:
#clf.fit(X, y.values.ravel())

In [156]:
#clf.best_params_
#{'max_depth': 15, 'max_features': 7, 'n_estimators': 1000, 'n_jobs': -1}



GBR
{'learning_rate': 0.03,
 'max_depth': 6,
 'min_samples_split': 4,
 'n_estimators': 350}
 
 {'learning_rate': 0.1,
 'max_depth': 6,
 'min_samples_split': 5,
 'n_estimators': 79}

forest =  RandomForestRegressor(bootstrap=True,
                                                   
                                                   criterion='mse',
                                                   max_depth=18, max_features=7,
                                                   max_leaf_nodes=None,
                                                   
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=2,
                                                   min_samples_split=4,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=1442,
                                                   n_jobs=-1, oob_score=False,
                                                   random_state=42, verbose=0,
                                                   warm_start=False)

In [157]:
forest = RandomForestRegressor(max_depth=19, 
                               max_features=6, 
                               n_estimators=400,
                               random_state=42)

In [166]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

##cv_score = cross_val_score(forest, X, y, scoring='r2', cv=kf, n_jobs = -1)
##print('R2: {:.5f}'.format(cv_score.mean()))

R2: 0.76287


In [159]:
#0.76469
#clf = ensemble.GradientBoostingRegressor(n_estimators=71, max_depth=6, min_samples_split=4,
#                                           learning_rate=0.1, loss='ls', random_state=42)

#clf = ensemble.GradientBoostingRegressor(n_estimators=79, max_depth=6, min_samples_split=5,
#                                           learning_rate=0.1, loss='ls', random_state=42)

#0.76489
clf = ensemble.GradientBoostingRegressor(n_estimators=353, max_depth=6, min_samples_split=5,
                                           learning_rate=0.03, loss='ls', random_state=42)
kf = KFold(n_splits=5, random_state=42, shuffle=True)


##cv_score = cross_val_score(clf, X, y, scoring='r2', cv=kf, n_jobs = -1)
##print('R2: {:.5f}'.format(cv_score.mean()))

In [160]:
from sklearn.ensemble import VotingRegressor



model = VotingRegressor([('Tree', forest), ('GBR', clf)])







In [161]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
cv_score = cross_val_score(model, X, y, scoring='r2', cv=kf, n_jobs = -1)
print('R2: {:.5f}'.format(cv_score.mean()))

R2: 0.77045


In [162]:
model.fit(X, y.values.ravel())

VotingRegressor(estimators=[('Tree',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=19, max_features=6,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=400,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                                                   warm_start=False)

In [163]:
y_pred = model.predict(X_test)

preds = pd.DataFrame()
preds['Id'] = X_test.index
preds['Price'] = y_pred

In [306]:
clf.fit(X, y.values.ravel())

y_pred = clf.predict(X_test)

preds = pd.DataFrame()
preds['Id'] = X_test.index
preds['Price'] = y_pred

In [88]:
forest.fit(X, y.values.ravel())

y_pred = forest.predict(X_test)

preds = pd.DataFrame()
preds['Id'] = X_test.index
preds['Price'] = y_pred

import matplotlib.pyplot as plt
%matplotlib inline
#style.use('fivethirtyeight')
#%matplotlib inline

def plot_f_importances(importances, X):    
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize = (20, 6))
    plt.title("Feature importances", fontsize=16)
    plt.bar(range(X.shape[1]), importances[indices] / importances.sum(),
           color="darkblue", align="center")
    plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 90, fontsize=14)
    plt.xlim([-1, X.shape[1]])

    plt.tight_layout()
    # plt.savefig('fe.jpg')
    plt.show()
    
plot_f_importances(importances = forest.feature_importances_, X=X)

import matplotlib.pyplot as plt
%matplotlib inline
#style.use('fivethirtyeight')
#%matplotlib inline

def plot_f_importances(importances, X):    
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize = (20, 6))
    plt.title("Feature importances", fontsize=16)
    plt.bar(range(X.shape[1]), importances[indices] / importances.sum(),
           color="darkblue", align="center")
    plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 90, fontsize=14)
    plt.xlim([-1, X.shape[1]])

    plt.tight_layout()
    # plt.savefig('fe.jpg')
    plt.show()
    
plot_f_importances(importances = clf.feature_importances_, X=X)

In [165]:
preds.to_csv('submission.csv', index=False)

In [164]:
preds

,Id,Price
0,725,157394.674439
1,15856,218168.989947
2,5480,286121.601287
3,15664,323893.214487
4,14275,142373.268306
...,...,...
4995,8180,242075.342545
4996,4695,128336.717088
4997,5783,336079.370918
4998,4780,199135.163111
